In [14]:
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, SVD, KNNBasic, NMF
from surprise import accuracy
from surprise import *
from surprise.model_selection.validation import cross_validate
import pickle

In [2]:
# Tải dữ liệu từ file CSV
df = pd.read_csv('Products_ThoiTrangNam_rating_raw.csv', sep='\t')

In [3]:
df.head()

,product_id,user_id,user,rating
0,190,1,karmakyun2nd,5
1,190,2,tranquangvinh_vv,5
2,190,3,nguyenquoctoan2005,5
3,190,4,nguyenthuyhavi,5
4,190,5,luonganh5595,5


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1024482 entries, 0 to 1024481
Data columns (total 4 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   product_id  1024482 non-null  int64 
 1   user_id     1024482 non-null  int64 
 2   user        1024482 non-null  object
 3   rating      1024482 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 31.3+ MB


In [5]:
reader = Reader()
data = Dataset.load_from_df(df[['user_id', 'product_id', 'rating']], reader)

## Xây dựng model Surprise với thuật toán SVD

In [6]:
algorithm = SVD()

In [7]:
results = cross_validate(algorithm, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8849  0.8755  0.8784  0.8799  0.8819  0.8801  0.0032  
MAE (testset)     0.5610  0.5567  0.5585  0.5588  0.5598  0.5590  0.0014  
Fit time          30.34   29.40   27.51   27.39   26.87   28.30   1.33    
Test time         3.83    2.17    2.86    2.91    2.71    2.90    0.54    


In [9]:
results

{'test_rmse': array([0.88491195, 0.87553444, 0.8783914 , 0.87993057, 0.88186381]),
 'test_mae': array([0.56095059, 0.55674581, 0.5585467 , 0.55879338, 0.55979332]),
 'fit_time': (30.3355712890625,
  29.401145935058594,
  27.509253978729248,
  27.385462760925293,
  26.8687801361084),
 'test_time': (3.826936721801758,
  2.1650373935699463,
  2.8601858615875244,
  2.9142343997955322,
  2.709207534790039)}

In [10]:
# If the results are OK => getting full dataset => fit model
trainset = data.build_full_trainset()
algorithm.fit(trainset)

## Sử dụng model SVD, gợi ý cho nhóm ID (ví dụ 199, 5, 1)

In [13]:
products_df = pd.read_csv('Products_ThoiTrangNam_raw.csv')
product_id_to_name = dict(zip(products_df['product_id'], products_df['product_name']))

def get_top_n_recommendations(algorithm, user_id, n=5):
    # Lấy danh sách tất cả product_id duy nhất từ df (giả sử df là dữ liệu đánh giá)
    all_product_ids = df['product_id'].unique()
    
    # Lấy danh sách product_id mà user đã đánh giá
    rated_product_ids = df[df['user_id'] == user_id]['product_id'].unique()
    
    # Dự đoán rating cho các sản phẩm chưa được đánh giá
    predictions = [algorithm.predict(user_id, product_id) for product_id in all_product_ids 
                   if product_id not in rated_product_ids]
    
    # Sắp xếp dự đoán theo giá trị rating dự đoán (estimated rating) giảm dần
    predictions.sort(key=lambda x: x.est, reverse=True)
    
    # Lấy top N product_id
    top_n_product_ids = [pred.iid for pred in predictions[:n]]
    
    # Chuyển đổi product_id sang product_name
    top_n_product_names = [product_id_to_name.get(pid, "Unknown Product") for pid in top_n_product_ids]
    
    return top_n_product_names

# Danh sách user_id cần gợi ý
user_ids = [199, 5, 1]

# Số lượng gợi ý cho mỗi user
n_recommendations = 5

# Đưa ra gợi ý cho từng user_id
for user_id in user_ids:
    recommendations = get_top_n_recommendations(algorithm, user_id, n_recommendations)
    print(f'Top {n_recommendations} gợi ý cho user {user_id}:')
    for i, product_name in enumerate(recommendations, 1):
        print(f'  {i}. {product_name}')

Top 5 gợi ý cho user 199:
  1. Áo tanktop nam nữ in hình City Cycle - áo ba lỗ sát nách unisex Local Brand
  2. Áo thun nam ngắn tay cổ chữ V khoét sâu thời trang
  3. Bộ ba lỗ mickey nam
  4. Áo ba lỗ tập Gym - Áo Tanktop cao cấp ONE ATHLETE - Logo nhũ vàng siêu chất - BiT Fitness TT.OA.LGTR
  5. [ Sale Sốc ] Áo hoodie Drew nỉ bông hàng cao cấp Ss2021
Top 5 gợi ý cho user 5:
  1. Áo GAP ba lỗ ,mặc thoáng mát ,thấm hút mồ hôi ,không nhăn không nhầu ,hàng full size
  2. Áo Lót Ba Lỗ Nam ARISTINO 100% Cotton, AC21, tag 85k
  3. Áo Ba Lỗ Nam Màu Trắng⭐⭐ JUVENO⭐⭐, co giãn 4 chiều, đủ size M-XXL - RSC01
  4. Áo sát nách thể thao gym bigsize cao cấp áo 3 lỗ tanktop nam xuất khẩu SNTT
  5. Áo ba lỗ nam thể thao mã [L02] chất thun lạnh co giãn 4 chiều hàng VNXK. Áo sát nách nam
Top 5 gợi ý cho user 1:
  1. Áo ba lỗ nam thun lạnh co giãn 4 chiều
  2. Áo Lót Ba Lỗ Nam ARISTINO 100% Cotton, AC21, tag 85k
  3. Áo Ba Lỗ Nam 100% Vải Poly Mềm Mỏng Siêu Thoáng Mát Thấm Hút Mồ Hôi Tốt Co Giãn 

In [15]:
# Sau khi huấn luyện model
with open('surprise_model.pkl', 'wb') as f:
    pickle.dump(algorithm, f)

In [16]:
# Lưu DataFrame gốc chứa đánh giá
df.to_csv('user_ratings.csv', index=False)

# Lưu từ điển ánh xạ product_id -> product_name
with open('product_id_to_name.pkl', 'wb') as f:
    pickle.dump(product_id_to_name, f)